In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
import requests
import json
import hvplot.pandas

# Import the API key
from config import geoapify_key

# Study data files
average_mort_path = "Data/average_mort_df.csv"
lat_lng_path = "Data/lat_lng_df .csv"

# Read the mouse data and the study results
average_mort = pd.read_csv(average_mort_path)
lat_lng = pd.read_csv(lat_lng_path)

# Combine the mortality data into a single DataFrame
world_df = pd.merge(average_mort, lat_lng, how="left", on=["Country Name", "Country Name"])


# Display the data table for preview
world_df

,Country Name,Average Mortality of 2019,Average Mortality of 2020,Average Mortality of 2021,Medical doctors (number),Average Immunization,Life expectancy total population,LNG,LAT
0,Japan,49.9290,50.0790,49.9895,311755.500000,97.800000,84.247228,138.252924,36.204824
1,Switzerland,43.4165,44.6305,44.9425,36183.600000,86.533333,83.567678,8.227512,46.818188
2,Singapore,44.3405,46.4585,44.7245,13371.400000,95.733333,83.050048,103.819836,1.352083
3,Sweden,47.9345,48.7595,47.9475,48191.200000,90.600000,82.523895,18.643501,60.128161
4,Luxembourg,52.5070,52.9335,51.3860,1706.333333,97.600000,82.409075,6.129583,49.815273
5,Nigeria,363.7190,367.1200,376.2795,79054.000000,52.066667,53.931194,8.675277,9.081999
6,Sierra Leone,236.3350,249.6125,248.1375,566.000000,87.933333,53.833848,-11.779889,8.460555
7,Chad,362.0845,376.6680,387.7775,709.800000,42.866667,53.706891,18.732207,15.454166
8,Lesotho,461.5370,449.0140,492.8125,998.000000,87.666667,52.931118,28.233608,-29.609988
9,Central African Republic,394.2650,407.9900,425.3465,329.500000,47.666667,52.170470,20.939444,6.611111


In [9]:
# Set the parameters for the type of search - Analyzing amount of hospitals in a 500 km radius of the Capital City of the Country 
radius = 500000

# Create an empty DataFrame to store the hospital data
hospital_df = pd.DataFrame(columns=["Country Name", "Latitude", "Longitude", "Hospital Count"])

# Iterate through the hotel_df DataFrame
for index, row in world_df.iterrows():

    # Get the country's name
    country = row["Country Name"]
    
    # Get latitude and longitude from the DataFrame
    latitude = row["LAT"]
    longitude = row["LNG"]
    
    # Set up a parameters dictionary
    params = {
        "limit": 150,
        "apiKey": geoapify_key,
        "format": "json",
        "categories": "healthcare.hospital",
        "filter": f"circle:{longitude},{latitude},{radius}",
        "bias": f"proximity:{longitude},{latitude}"
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    data = response.json()

    # Count the number of hospitals in the response
    hospital_count = len(data["features"])

    # Append the data to the hospital_df DataFrame
    hospital_df = hospital_df.append({"Country Name": country, "Latitude": latitude, "Longitude": longitude, "Hospital Count": hospital_count}, ignore_index=True)

# Display the hospital_df DataFrame
hospital_df


C:\Users\Julia\AppData\Local\Temp\ipykernel_11816\1446679455.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hospital_df = hospital_df.append({"Country Name": country, "Latitude": latitude, "Longitude": longitude, "Hospital Count": hospital_count}, ignore_index=True)
C:\Users\Julia\AppData\Local\Temp\ipykernel_11816\1446679455.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hospital_df = hospital_df.append({"Country Name": country, "Latitude": latitude, "Longitude": longitude, "Hospital Count": hospital_count}, ignore_index=True)
C:\Users\Julia\AppData\Local\Temp\ipykernel_11816\1446679455.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hospital_df = hospital_df.append({"Country Name": country, "Latitude": la

,Country Name,Latitude,Longitude,Hospital Count
0,Japan,36.204824,138.252924,42
1,Switzerland,46.818188,8.227512,10
2,Singapore,1.352083,103.819836,133
3,Sweden,60.128161,18.643501,0
4,Luxembourg,49.815273,6.129583,15
5,Nigeria,9.081999,8.675277,0
6,Sierra Leone,8.460555,-11.779889,14
7,Chad,15.454166,18.732207,0
8,Lesotho,-29.609988,28.233608,4
9,Central African Republic,6.611111,20.939444,4


In [11]:
#Merge the data from the outline.df and the hospital.df
final_df = pd.merge(hospital_df, world_df, how="left", on=["Country Name", "Country Name"])
final_df = final_df.sort_values(by=['Life expectancy total population'], ascending=False)
# Create a list of column we want to analyze
columns_to_analyze = ['Country Name','LNG', 'LAT', 'Hospital Count', 'Life expectancy total population', 'Average Immunization', 'Medical doctors (number)']

# Extract the specified columns into a new DataFrame
final_complete = final_df[columns_to_analyze]

final_complete

,Country Name,LNG,LAT,Hospital Count,Life expectancy total population,Average Immunization,Medical doctors (number)
0,Japan,138.252924,36.204824,42,84.247228,97.800000,311755.500000
1,Switzerland,8.227512,46.818188,10,83.567678,86.533333,36183.600000
2,Singapore,103.819836,1.352083,133,83.050048,95.733333,13371.400000
3,Sweden,18.643501,60.128161,0,82.523895,90.600000,48191.200000
4,Luxembourg,6.129583,49.815273,15,82.409075,97.600000,1706.333333
5,Nigeria,8.675277,9.081999,0,53.931194,52.066667,79054.000000
6,Sierra Leone,-11.779889,8.460555,14,53.833848,87.933333,566.000000
7,Chad,18.732207,15.454166,0,53.706891,42.866667,709.800000
8,Lesotho,28.233608,-29.609988,4,52.931118,87.666667,998.000000
9,Central African Republic,20.939444,6.611111,4,52.170470,47.666667,329.500000


In [12]:
%%capture --no-display

# Configure the map plot
map_plot = final_complete.hvplot.points(
    "LAT",
    "LNG",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=700,
    size="Life expectancy total population",
    color = "Country Name",
    hover_cols=["Hospital Count", "Average Immunization", "Medical doctors (number)"]
).opts(title="Plot Title")

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LAT,LNG]   (Country Name,Life expectancy total population,Hospital Count,Average Immunization,Medical doctors (number))